# Tech Layoffs Data Cleaning in SQL

## Project Goal: Workforce Risk Analysis for Strategic Planning

This project analyzes global tech layoffs from 2020–2024 to uncover patterns across companies, industries, and countries. It’s designed for **HR teams, workforce planners, investors, and policy makers** who need fast, data-driven insights to guide hiring, scaling, and reskilling decisions.

Whether you're planning headcount, evaluating sector risk, or designing support programs, this analysis helps you act with clarity and confidence.

I used **SQL (SQLite3)** for data cleaning and trend analysis, and built an **interactive Tableau dashboard** to visualize layoffs by geography, industry, and company.

> *Disclaimer: This dataset is compiled from public sources and may not reflect all global layoffs. It’s intended for exploratory analysis and strategic pattern recognition—not for forecasting or operational decisions.*

## Business Impact Summary

- Identified high-risk sectors and regions for workforce disruption  
- Revealed layoff timing trends to inform hiring and investment cycles  
- Delivered role-specific recommendations for strategic planning


For those who want to explore the work on GitHub: 

👉 [Tech Layoffs 2020–2024 on GitHub](https://github.com/Wilfrida-Were/Tech-Layoffs-Data-Cleaning-in-SQL/blob/main/README.md)

## 📑 Table of Contents
* [Interactive Dashboard](#interactive-dashboard)
* [Code](#code)
* [Key Insights](#key-insights)
* [Recommendations](#recommendations)
* [Client Scenario: Applying the Recommendations](#client-scenario)
* [Who Can Use This Project](#who-can-use-this-project)
* [Tech Stack](#tech-stack)
* [Workflow](#workflow)
* [Key Learnings](#key-learnings)
* [Let’s Connect](#lets-connect)


<a id="interactive-dashboard"></a>
## 🚀 Interactive Dashboard
👉 [View the Interactive Dashboard on Tableau Public](https://public.tableau.com/app/profile/wilfrida.were/viz/TechLayoffs2020-2024_17198461868990/TECHLAYOFFS2020-2024)  

Preview (click image to open dashboard):  
[![Tech Layoffs Dashboard](https://raw.githubusercontent.com/Wilfrida-Were/Tech-Layoffs-Data-Cleaning-in-SQL/7204e10b6aff469807b4e416e4237384e33482f1/TECH%20LAYOFFS%202020-2024.png)](https://public.tableau.com/app/profile/wilfrida.were/viz/TechLayoffs2020-2024_17198461868990/TECHLAYOFFS2020-2024)  

<a id="code"></a>
## 💻 Code

In [1]:
# ============================================================
# 📚 IMPORT LIBRARIES
# ============================================================

# Core data manipulation
import numpy as np
import pandas as pd

# Database (SQL) utilities
import sqlite3   # for creating and working with an SQLite database using SQL commands

In [2]:
# ============================================================
# 📂 LOAD DATA
# ============================================================

# Read the Excel dataset into a DataFrame
tech_layoffs = pd.read_excel('/kaggle/input/tech-layoffs-2020-2024/tech_layoffs.xlsx')


# Create a working copy to preserve the original data
df = tech_layoffs.copy()

# Preview the first 5 rows of the dataset
df.head()
df = tech_layoffs.copy() # make a copy
df.head()

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18,10.0,1000.0,900.0,Retail,Unknown,NaN,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.77493,-122.41942


In [3]:
# Preview dataset structure
df.info()  # 1672 rows and 16 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672 entries, 0 to 1671
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   #                            1672 non-null   int64         
 1   Company                      1672 non-null   object        
 2   Location_HQ                  1672 non-null   object        
 3   Country                      1672 non-null   object        
 4   Continent                    1672 non-null   object        
 5   Laid_Off                     1565 non-null   float64       
 6   Date_layoffs                 1672 non-null   datetime64[ns]
 7   Percentage                   1570 non-null   float64       
 8   Company_Size_before_Layoffs  1511 non-null   float64       
 9   Company_Size_after_layoffs   1536 non-null   float64       
 10  Industry                     1672 non-null   object        
 11  Stage                        1672 non-null 

The columns `Laid_Off`, `Percentage`, `Company_Size_before_Layoffs`, `Company_Size_after_layoffs` have missing values so I will inspect them. This helps decide whether to impute or drop those rows with missing values. I don't need the `Money_Raised_in_$_mil` column

## Connect to a Database and convert the Dataframe to a Table

In [4]:
# ============================================================
# 🗄️ CONNECT TO SQLITE DATABASE
# ============================================================

# Establish a connection to a local SQLite database file.
# - If the database file does not exist, it will be created.
# - This connection allows us to run SQL commands directly.

# Connect to the database
conn = sqlite3.connect('tech_layoffs_data.db')

# Create 'tech_layoffs' table
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='tech_layoffs'")
table_exists = len(cursor.fetchall()) > 0

if not table_exists:
  # Load data to 'tech_layoffs' table only if it doesn't exist
  df.to_sql('tech_layoffs', conn, index=False)
  print("Table 'tech_layoffs' created and data loaded successfully.")
else:
  print("Table 'tech_layoffs' already exists. Skipping data loading.")

Table 'tech_layoffs' created and data loaded successfully.


In [5]:
# ============================================================
# INSPECT FIRST 10 ROWS USING SQL
# ============================================================

# Query the first 10 rows from the 'tech_layoffs' table
# - pd.read_sql_query() integrates SQL queries directly into pandas
# - Useful for inspecting the raw data inside the SQLite database

first_10_rows = pd.read_sql_query("""SELECT *
                                   FROM tech_layoffs
                                   LIMIT 10;""", conn)
first_10_rows

,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20 00:00:00,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19 00:00:00,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18 00:00:00,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18 00:00:00,10.0,1000.0,900.0,Retail,Unknown,None,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14 00:00:00,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.77493,-122.41942
5,16,Bolt,San Francisco Bay Area,USA,North America,130.0,2023-12-14 00:00:00,29.0,450.0,320.0,Finance,Series E,$1300,2023,37.77493,-122.41942
6,20,Invitae,San Francisco Bay Area,USA,North America,235.0,2023-12-13 00:00:00,15.0,1567.0,1332.0,Healthcare,Post-IPO,None,2023,37.77493,-122.41942
7,21,Etsy,New York City,USA,North America,225.0,2023-12-13 00:00:00,11.0,2045.0,1820.0,Retail,Post-IPO,$97,2023,40.71427,-74.00597
8,27,Chipper Cash,San Francisco Bay Area,USA,North America,15.0,2023-12-11 00:00:00,33.0,45.0,30.0,Finance,Series C,$302,2023,37.77493,-122.41942
9,31,Zulily,Seattle,USA,North America,839.0,2023-12-08 00:00:00,100.0,839.0,0.0,Retail,Acquired,$194,2023,47.60621,-122.33207


## Data Cleaning

## Identify Spelling Errors

In [6]:
# ============================================================
# DATA CLEANING — IDENTIFY SPELLING ERRORS
# ============================================================

# Inspect unique values in the 'Industry' column
# - Helps detect spelling mistakes or inconsistent labels
# - Ensures categories can be standardized later
# Inspect the Industry column 

industry = pd.read_sql_query("""
SELECT DISTINCT Industry
FROM tech_layoffs;
""",conn)

industry

,Industry
0,Consumer
1,Healthcare
2,Energy
3,Retail
4,Transportation
5,Finance
6,Product
7,Media
8,Other
9,AI


## Update spelling mistakes in the Industry column

These industries will be **updated**:

* Transport… &#8594; Transportation
* Manufact… &#8594; Manufacturing
* Infrastruc… &#8594; Infrastructure
* Construct… &#8594; Construction

In [7]:
# ============================================================
# DATA CLEANING — FIX SPELLING MISTAKES (Industry Column)
# ============================================================

# Standardize Industry names:
# - Replace variations like 'Transport', 'Transport Services', etc. with a consistent label 'Transportation'

# Update to Transportation
update_query = """
UPDATE tech_layoffs
SET Industry = 'Transportation'
WHERE Industry LIKE 'Transport%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

# ------------------------------------------------------------
# 🔍 VERIFY: Check distinct Industry values after update
# ------------------------------------------------------------
industry_check = pd.read_sql_query("""
    SELECT DISTINCT Industry
    FROM tech_layoffs
    ORDER BY Industry;
""", conn)

display(industry_check)

,Industry
0,AI
1,Aerospace
2,Construction
3,Construct…
4,Consumer
5,Crypto
6,Data
7,Education
8,Energy
9,Finance


In [8]:
# ============================================================
# DATA CLEANING — FIX SPELLING MISTAKES (Industry Column)
# ============================================================

# Standardize Industry names:
# - Replace variations like 'Manufact' etc with a consistent label 'Manufacturing'

# Update to Manufacturing
update_query = """
UPDATE tech_layoffs
SET Industry = 'Manufacturing'
WHERE Industry LIKE 'Manufact%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

# ------------------------------------------------------------
# 🔍 VERIFY: Check distinct Industry values after update
# ------------------------------------------------------------
industry_check = pd.read_sql_query("""
    SELECT DISTINCT Industry
    FROM tech_layoffs
    ORDER BY Industry;
""", conn)

display(industry_check)

,Industry
0,AI
1,Aerospace
2,Construction
3,Construct…
4,Consumer
5,Crypto
6,Data
7,Education
8,Energy
9,Finance


In [9]:
# ============================================================
# DATA CLEANING — FIX SPELLING MISTAKES (Industry Column)
# ============================================================

# Standardize Industry names:
# - Replace variations like 'Infrastr' etc with a consistent label 'Infrastructure'

# Update to Infrastructure
update_query = """
UPDATE tech_layoffs
SET Industry = 'Infrastructure'
WHERE Industry LIKE 'Infrastruc%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

# ------------------------------------------------------------
# 🔍 VERIFY: Check distinct Industry values after update
# ------------------------------------------------------------
industry_check = pd.read_sql_query("""
    SELECT DISTINCT Industry
    FROM tech_layoffs
    ORDER BY Industry;
""", conn)

display(industry_check)

,Industry
0,AI
1,Aerospace
2,Construction
3,Construct…
4,Consumer
5,Crypto
6,Data
7,Education
8,Energy
9,Finance


In [10]:
# ============================================================
# DATA CLEANING — FIX SPELLING MISTAKES (Industry Column)
# ============================================================

# Standardize Industry names:
# - Replace variations like 'Construct' etc with a consistent label 'Construction'

# Update to Construction
update_query = """
UPDATE tech_layoffs
SET Industry = 'Construction'
WHERE Industry LIKE 'Construct%';
"""

# Execute the update query using conn.execute()
conn.execute(update_query)

# ------------------------------------------------------------
# 🔍 VERIFY: Check distinct Industry values after update
# ------------------------------------------------------------
industry_check = pd.read_sql_query("""
    SELECT DISTINCT Industry
    FROM tech_layoffs
    ORDER BY Industry;
""", conn)

display(industry_check)

,Industry
0,AI
1,Aerospace
2,Construction
3,Consumer
4,Crypto
5,Data
6,Education
7,Energy
8,Finance
9,Fitness


## Duplicate Rows

In [11]:
# ============================================================
# DATA CLEANING — IDENTIFY DUPLICATE ROWS
# ============================================================

# - Detect duplicate records that repeat across the following columns: Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry
# - Use ROW_NUMBER() with PARTITION BY to assign a sequence number for duplicates within each group of identical values.

duplicate_rows = pd.read_sql_query("""
SELECT *
FROM (
  SELECT Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry,
         ROW_NUMBER() OVER (PARTITION BY Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry) AS row_num
  FROM tech_layoffs
) AS duplicates  
WHERE row_num > 1;
""", conn)

duplicate_rows

,Company,Location_HQ,Country,Laid_Off,Date_layoffs,Industry,row_num
0,Beyond Meat,Los Angeles,USA,200.0,2022-10-14 00:00:00,Food,2
1,Cazoo,London,United Kingdom,750.0,2022-06-07 00:00:00,Transportation,2
2,Chipper Cash,San Francisco Bay Area,USA,15.0,2023-12-11 00:00:00,Finance,2
3,Cruise,San Francisco Bay Area,USA,900.0,2023-12-14 00:00:00,Transportation,2
4,Enphase Energy,San Francisco Bay Area,USA,350.0,2023-12-18 00:00:00,Energy,2
5,Etsy,New York City,USA,225.0,2023-12-13 00:00:00,Retail,2
6,Invitae,San Francisco Bay Area,USA,235.0,2023-12-13 00:00:00,Healthcare,2
7,Oda,Oslo,Norway,70.0,2022-11-01 00:00:00,Food,2
8,ShareChat,Bengaluru,India,200.0,2023-12-20 00:00:00,Consumer,2
9,Udaan,Bengaluru,India,100.0,2023-12-18 00:00:00,Retail,2


In [12]:
# ============================================================
# INSPECT DUPLICATE EXAMPLE — BEYOND MEAT
# ============================================================

# - Investigate one of the identified duplicate companies ("Beyond Meat").
# - Select its details (Company, HQ location, Country, Layoffs, Date, Industry) to confirm and visually inspect where duplication occurs.

Beyond_meat = pd.read_sql_query("""
SELECT Company, Location_HQ, Country, Laid_Off,Date_layoffs, Industry
FROM tech_layoffs
WHERE Company LIKE 'Beyond Meat';
""",conn)

Beyond_meat

,Company,Location_HQ,Country,Laid_Off,Date_layoffs,Industry
0,Beyond Meat,Los Angeles,USA,65.0,2023-11-02 00:00:00,Food
1,Beyond Meat,Los Angeles,USA,200.0,2022-10-14 00:00:00,Food
2,Beyond Meat,Los Angeles,USA,200.0,2022-10-14 00:00:00,Food
3,Beyond Meat,Los Angeles,USA,40.0,2022-08-03 00:00:00,Food


## Missing Values

In [13]:
# ============================================================
# MISSING VALUES CHECK
# ============================================================

# - Identify rows where critical columns have missing values.
# - Columns checked: Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs.
# - Helps assess data completeness before cleaning/imputation.

missing_values = pd.read_sql_query("""
SELECT Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs
FROM tech_layoffs
WHERE Laid_Off IS NULL
OR Percentage IS NULL 
OR Company_Size_before_Layoffs IS NULL 
OR Company_Size_after_layoffs IS NULL;
""",conn)

for col in missing_values.columns:
    print(f"Missing values in '{col}': {missing_values[col].isnull().sum()}")

print() # blank line
missing_values.head(10) # the first 10 rows

Missing values in 'Laid_Off': 107
Missing values in 'Percentage': 102
Missing values in 'Company_Size_before_Layoffs': 161
Missing values in 'Company_Size_after_layoffs': 136



,Laid_Off,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs
0,NaN,NaN,None,NaN
1,NaN,NaN,None,NaN
2,100.0,NaN,None,0.0
3,NaN,20.0,None,NaN
4,NaN,5.0,None,NaN
5,54.0,NaN,None,NaN
6,20.0,NaN,None,NaN
7,NaN,100.0,None,0.0
8,NaN,NaN,None,NaN
9,NaN,40.0,None,NaN


## The Problem with Imputation

In this case, the columns `Laid_Off`, `Percentage`, `Company_Size_before_Layoffs`, and `Company_Size_after_layoffs` are all **related**.

Imagine these rows represent companies that went through layoffs.  Since we don't have all the information *(number laid off, company size before/after)*,  trying to guess those missing values would be like making up a story.  It's better to remove these rows and focus on the companies where we have complete data for a more accurate analysis.

In [14]:
# ============================================================
# PROBLEM WITH IMPUTATION → FILTERING CLEAN ROWS
# ============================================================

# - Instead of imputing missing values (which may introduce bias or false assumptions),we filter out rows that contain NULL values in critical columns.
# - Columns checked: Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs.
# - This ensures we only work with complete and reliable records.

Cleaned_tech_layoffs = pd.read_sql_query("""
SELECT *
FROM tech_layoffs
WHERE Laid_Off IS NOT NULL
AND Percentage IS NOT NULL 
AND Company_Size_before_Layoffs IS NOT NULL 
AND Company_Size_after_layoffs IS NOT NULL;
""",conn)

print(Cleaned_tech_layoffs.shape) # 1511 clean rows remain after Filtering 
print() # blank line
Cleaned_tech_layoffs.head()

(1511, 16)



,#,Company,Location_HQ,Country,Continent,Laid_Off,Date_layoffs,Percentage,Company_Size_before_Layoffs,Company_Size_after_layoffs,Industry,Stage,Money_Raised_in_$_mil,Year,lat,lng
0,3,ShareChat,Bengaluru,India,Asia,200.0,2023-12-20 00:00:00,15.0,1333.0,1133.0,Consumer,Series H,$1700,2023,12.97194,77.59369
1,4,InSightec,Haifa,Israel,Asia,100.0,2023-12-19 00:00:00,20.0,500.0,400.0,Healthcare,Unknown,$733,2023,32.81841,34.98850
2,6,Enphase Energy,San Francisco Bay Area,USA,North America,350.0,2023-12-18 00:00:00,10.0,3500.0,3150.0,Energy,Post-IPO,$116,2023,37.54827,-121.98857
3,7,Udaan,Bengaluru,India,Asia,100.0,2023-12-18 00:00:00,10.0,1000.0,900.0,Retail,Unknown,None,2023,12.97194,77.59369
4,14,Cruise,San Francisco Bay Area,USA,North America,900.0,2023-12-14 00:00:00,24.0,3750.0,2850.0,Transportation,Acquired,$15000,2023,37.77493,-122.41942


In [15]:
# ============================================================
# REMOVE DUPLICATE ROWS
# ============================================================

# - Drop duplicate records to ensure dataset integrity.
# - Duplicates are identified based on: Company, Location_HQ, Country, Laid_Off, Date_layoffs, and Industry.
# - We keep only the *first* occurrence of each duplicate group.

Cleaned_tech_layoffs = Cleaned_tech_layoffs.drop_duplicates(subset=['Company', 'Location_HQ', 'Country', 'Laid_Off',
                                                                    'Date_layoffs','Industry'], keep='first') # keep first instances

print(Cleaned_tech_layoffs.shape) # 1501 rows remain after dropping duplicate rows

(1501, 16)


In [16]:
# Save the cleaned DataFrame to a CSV file
Cleaned_tech_layoffs.to_csv('Cleaned_tech_layoffs.csv', index=False)
print('Cleaned_tech_layoffs.csv saved.')

Cleaned_tech_layoffs.csv saved.


In [17]:
# ============================================================
# CLOSE DATABASE CONNECTION
# ============================================================

# Commit any pending changes to the database
conn.commit()

# Properly close the SQLite connection
conn.close()

<a id="key-insights"></a>
## 🔑 Key Insights

- **📉 Total Layoffs:** 409,323 employees affected worldwide.  
- **🌍 By Country:**  
  - USA dominates with ~ 305K layoffs (~74%).  
  - Other impacted countries: India (27K), Germany (23K), and the UK (13K).  
- **📈 Trend Over Time:**  
  - Lowest layoffs: 2021 (~6.7K).  
  - Peak: 2023 (174K).  
  - Sharp decline in 2024 (~49K).  
- **🏢 By Company:** Amazon (27K), Meta (21K), and Google (12K) top the list, followed by SAP and Microsoft (10K each).  
- **🏭 By Industry:** Retail (56K) and Consumer tech (55K) are most affected, followed by Transportation (34K) and Food tech (30K).  
- **⚠️ Extreme Cases:** Startups like Zulily, ZestMoney, Udayy, and TutorMundi laid off **100% of their staff** in specific years.  


<a id="recommendations"></a>
## ✅ Recommendations

**For HR & Workforce Planners**  
- Avoid aggressive hiring in volatile sectors like Retail and Consumer Tech.  
- Prioritize reskilling programs in regions hit hardest (e.g. USA, India).  
- Use layoff timing trends to guide headcount planning and contract renewals.

**For Investors & Analysts**  
- Reassess exposure to companies with repeated or large-scale layoffs (e.g. Amazon, Meta).  
- Diversify into industries with lower layoff rates like Finance or Hardware.  
- Monitor startup risk—100% layoffs signal fragility in early-stage ventures.

**For Founders & Business Leaders**  
- Use industry-level layoff data to inform conservative hiring and flexible staffing.  
- Benchmark against peers to assess operational resilience.  
- Prepare contingency plans for sectors with high churn.

**For Policy Makers & NGOs**  
- Target workforce support in countries with high layoff volumes (USA, India).  
- Design training programs for displaced workers in Retail and Consumer Tech.  
- Track startup closures to inform entrepreneurship support policies.

<a id="client-scenario"></a>
## 🧩 Client Scenario: Applying the Recommendations

Let’s say you’re an **HR Director at a global tech firm** planning 2024 hiring and retention strategy.

### 🎯 Goal: Reduce Workforce Risk in Volatile Sectors

- Review layoff trends by industry—Retail and Consumer Tech show highest churn  
- Pause hiring in those sectors; shift focus to stable areas like Finance and Hardware  
- Launch internal reskilling programs for roles vulnerable to disruption  
- Use Tableau dashboard to present layoff timing trends to leadership

**Why it works:**  
This approach helps avoid over-hiring in unstable sectors, supports employee retention, and builds resilience through proactive planning.


### 💼 Goal: Advise Leadership on Strategic Talent Allocation

- Highlight peak layoff year (2023) and sharp decline in 2024  
- Show country-level impact—USA and India are most affected  
- Recommend flexible staffing models and regional hiring adjustments  
- Use company-level data to benchmark against competitors

**Why it works:**  
Grounding decisions in macro-level data builds trust and credibility. It helps leadership align talent strategy with market realities.

<a id="who-can-use-this-project"></a>
## 👥 Who Can Use This Project?

- **Job Seekers**  
  → Target stable industries and regions based on historical layoff trends.

- **Investors & Analysts**  
  → Assess sector-level risk and identify macro patterns for smarter decisions.

- **Founders & Business Leaders**  
  → Benchmark industry performance to guide hiring, scaling, or contingency planning.

- **Policy Makers**  
  → Design targeted reskilling and support programs for affected sectors and regions.

- **Educators & Learners**  
  → Explore SQL workflows and dashboards as a real-world case study in data analysis and business strategy.

<a id="tech-stack"></a>
## ⚙️ Tech Stack
- **SQL (SQLite3)** – Data cleaning and preprocessing  
- **Tableau** – Interactive dashboards and visualization  
- **Kaggle Notebook** – End-to-end workflow with code and explanations  

<a id="workflow"></a>
## 🔄 Workflow
1. **Raw Data** → [Kaggle Dataset: Tech Layoffs 2020–2024](https://www.kaggle.com/datasets/ulrikeherold/tech-layoffs-2020-2024)  
2. **Data Cleaning** → Performed with SQL (SQLite3).  
3. **Exploration** → SQL queries for aggregation & trend analysis.  
4. **Visualization** → Built Tableau dashboard for insights and storytelling. 

<a id="key-learnings"></a>
## 📌 Key Learnings
- Handling missing and inconsistent data in SQL.  
- Structuring cleaned datasets for visualization.  
- Designing dashboards that highlight **business-relevant insights**. 

<a id="lets-connect"></a>
## 🔗 Let’s Connect

Feel free to connect, follow, or support:  

[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect-blue?style=flat&logo=linkedin)](https://linkedin.com/in/wilfridawere/)  

[![Twitter](https://img.shields.io/badge/X-Follow-black?style=flat&logo=twitter)](https://x.com/wilfridawere)  

[![Website](https://img.shields.io/badge/Website-Visit-orange?style=flat&logo=google-chrome)](https://www.wilfridawere.com/)  

[![Kaggle](https://img.shields.io/badge/Kaggle-Follow-blue?style=flat&logo=kaggle)](https://kaggle.com/wilfridawere)  

[![GitHub](https://img.shields.io/badge/GitHub-Projects-black?style=flat&logo=github)](https://github.com/Wilfrida-Were)  

[![Ko-fi](https://img.shields.io/badge/Ko--fi-Support-red?style=flat&logo=ko-fi)](https://ko-fi.com/wilfridawere/)